In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from grouper import group_by_volume, group_by_time
from pyfolio.timeseries import perf_stats
from functools import partial
import sys
sys.path.append('/home/tomek/ib_tools')
from indicators import get_ATR, get_signals
%matplotlib inline

In [2]:
df = pd.read_csv('/home/tomek/ib_tools/blotter/backtest_2019-10-31_15-17.csv')
df.time = df.time.astype('datetime64')
df.set_index('time', inplace=True)
df

,sys_time,contract,action,amount,price,exec_ids,order_id,reason,com_exec_id,commission,currency,realizedPNL,com_sys_time
time,,,,,,,,,,,,,
2018-02-09 00:50:00,2019-10-31 15:17:25.970319,NQZ9,SELL,1,6304.175,[1],1,entry,1,1.3,USD,0.0,2019-10-31 15:17:25.970586
2018-02-09 01:15:00,2019-10-31 15:17:25.973439,NQZ9,BUY,1,6346.150,[2],2,stop-out,2,1.3,USD,-839.5,2019-10-31 15:17:25.973507
2018-02-09 16:50:00,2019-10-31 15:17:26.522056,NQZ9,SELL,1,6300.250,[3],3,entry,3,1.3,USD,0.0,2019-10-31 15:17:26.522312
2018-02-09 16:55:00,2019-10-31 15:17:26.522956,NQZ9,BUY,1,6348.625,[4],4,stop-out,4,1.3,USD,-967.5,2019-10-31 15:17:26.523001
2018-02-09 18:05:00,2019-10-31 15:17:26.687478,NQZ9,SELL,1,6280.100,[5],5,entry,5,1.3,USD,0.0,2019-10-31 15:17:26.687729
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-09-03 16:16:00,2019-10-31 15:40:36.640546,NQZ9,SELL,1,7609.825,[871],1002,entry,871,1.3,USD,0.0,2019-10-31 15:40:36.640936
2019-09-03 16:41:00,2019-10-31 15:40:36.804398,NQZ9,BUY,1,7639.275,[872],1003,stop-out,872,1.3,USD,-589.0,2019-10-31 15:40:36.804467
2019-09-03 19:51:00,2019-10-31 15:40:38.419814,NQZ9,SELL,1,7596.550,[873],1004,entry,873,1.3,USD,0.0,2019-10-31 15:40:38.420322


In [3]:
df[df.reason == 'entry'].price.count()

438

In [4]:
df[df.reason != 'entry'].price.count()

437

In [5]:
df.price.count()

875

In [6]:
df[df.realizedPNL > 4000]

,sys_time,contract,action,amount,price,exec_ids,order_id,reason,com_exec_id,commission,currency,realizedPNL,com_sys_time
time,,,,,,,,,,,,,
2018-06-07 16:40:00,2019-10-31 15:19:14.978665,NQZ9,SELL,1,7176.69933,[162],187,close,162,1.3,USD,4014.49,2019-10-31 15:19:14.978871
2018-10-11 15:33:00,2019-10-31 15:22:11.868256,NQZ9,BUY,1,7072.07500,[322],369,close,322,1.3,USD,6349.00,2019-10-31 15:22:11.868524
2018-11-20 16:57:00,2019-10-31 15:24:00.299635,NQZ9,BUY,1,6606.15000,[430],493,close,430,1.3,USD,4284.50,2019-10-31 15:24:00.299901
2018-11-29 16:31:00,2019-10-31 15:24:22.367062,NQZ9,SELL,1,6885.30000,[442],506,close,442,1.3,USD,5470.00,2019-10-31 15:24:22.367334
2018-12-06 16:29:00,2019-10-31 15:24:39.328556,NQZ9,BUY,1,6708.57500,[448],514,close,448,1.3,USD,4920.00,2019-10-31 15:24:39.328818
2019-02-01 01:03:00,2019-10-31 15:27:23.110750,NQZ9,SELL,1,6896.60000,[538],622,close,538,1.3,USD,4117.50,2019-10-31 15:27:23.111404
2019-06-10 21:59:00,2019-10-31 15:34:49.803499,NQZ9,SELL,1,7509.05000,[730],841,close,730,1.3,USD,5025.50,2019-10-31 15:34:49.803826


In [7]:
df.tail(50)

,sys_time,contract,action,amount,price,exec_ids,order_id,reason,com_exec_id,commission,currency,realizedPNL,com_sys_time
time,,,,,,,,,,,,,
2019-08-09 21:55:00,2019-10-31 15:38:48.318872,NQZ9,SELL,1,7659.400,[826],952,stop-out,826,1.3,USD,-558.0,2019-10-31 15:38:48.318952
2019-08-12 09:24:00,2019-10-31 15:38:50.167828,NQZ9,BUY,1,7706.675,[827],953,entry,827,1.3,USD,0.0,2019-10-31 15:38:50.168342
2019-08-12 10:07:00,2019-10-31 15:38:50.256128,NQZ9,SELL,1,7677.600,[828],954,stop-out,828,1.3,USD,-581.5,2019-10-31 15:38:50.256196
2019-08-12 13:20:00,2019-10-31 15:38:51.081765,NQZ9,SELL,1,7599.725,[829],955,entry,829,1.3,USD,0.0,2019-10-31 15:38:51.082146
2019-08-12 16:32:00,2019-10-31 15:38:52.279831,NQZ9,BUY,1,7628.600,[830],956,stop-out,830,1.3,USD,-577.5,2019-10-31 15:38:52.279901
2019-08-12 20:28:00,2019-10-31 15:38:54.213105,NQZ9,SELL,1,7565.775,[831],957,entry,831,1.3,USD,0.0,2019-10-31 15:38:54.213480
2019-08-13 15:36:00,2019-10-31 15:38:58.455629,NQZ9,BUY,1,7594.275,[832],958,stop-out,832,1.3,USD,-570.0,2019-10-31 15:38:58.455827
2019-08-13 15:38:00,2019-10-31 15:38:58.570325,NQZ9,BUY,1,7619.625,[833],959,entry,833,1.3,USD,0.0,2019-10-31 15:38:58.570810
2019-08-14 12:18:00,2019-10-31 15:39:04.911309,NQZ9,SELL,1,7670.025,[834],961,close,834,1.3,USD,1008.0,2019-10-31 15:39:04.911706


In [8]:
df.loc['20190829':].head(50)

,sys_time,contract,action,amount,price,exec_ids,order_id,reason,com_exec_id,commission,currency,realizedPNL,com_sys_time
time,,,,,,,,,,,,,
2019-08-29 03:10:00,2019-10-31 15:40:17.373393,NQZ9,SELL,1,7568.950,[864],994,stop-out,864,1.3,USD,-549.5,2019-10-31 15:40:17.373464
2019-08-29 09:14:00,2019-10-31 15:40:18.479986,NQZ9,BUY,1,7655.175,[865],995,entry,865,1.3,USD,0.0,2019-10-31 15:40:18.480372
2019-08-30 17:06:00,2019-10-31 15:40:27.040332,NQZ9,SELL,1,7684.450,[866],997,close,866,1.3,USD,585.5,2019-10-31 15:40:27.040702
2019-09-02 02:02:00,2019-10-31 15:40:29.661950,NQZ9,SELL,1,7630.725,[867],998,entry,867,1.3,USD,0.0,2019-10-31 15:40:29.662468
2019-09-02 09:19:00,2019-10-31 15:40:30.811884,NQZ9,BUY,1,7656.750,[868],999,stop-out,868,1.3,USD,-520.5,2019-10-31 15:40:30.811956
2019-09-02 17:59:00,2019-10-31 15:40:32.467934,NQZ9,SELL,1,7626.175,[869],1000,entry,869,1.3,USD,0.0,2019-10-31 15:40:32.468304
2019-09-03 03:37:00,2019-10-31 15:40:33.294572,NQZ9,BUY,1,7655.050,[870],1001,stop-out,870,1.3,USD,-577.5,2019-10-31 15:40:33.294643
2019-09-03 16:16:00,2019-10-31 15:40:36.640546,NQZ9,SELL,1,7609.825,[871],1002,entry,871,1.3,USD,0.0,2019-10-31 15:40:36.640936
2019-09-03 16:41:00,2019-10-31 15:40:36.804398,NQZ9,BUY,1,7639.275,[872],1003,stop-out,872,1.3,USD,-589.0,2019-10-31 15:40:36.804467


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 875 entries, 2018-02-09 00:50:00 to 2019-09-04 05:04:00
Data columns (total 13 columns):
sys_time        875 non-null object
contract        875 non-null object
action          875 non-null object
amount          875 non-null int64
price           875 non-null float64
exec_ids        875 non-null object
order_id        875 non-null int64
reason          875 non-null object
com_exec_id     875 non-null int64
commission      875 non-null float64
currency        875 non-null object
realizedPNL     875 non-null float64
com_sys_time    875 non-null object
dtypes: float64(3), int64(3), object(7)
memory usage: 135.7+ KB


In [10]:
df.realizedPNL.sum()

27158.6

In [11]:
# percentage of winning transactions
round(df[df.realizedPNL>0].realizedPNL.count() / df[df.realizedPNL != 0].realizedPNL.count(), 2)

0.26

In [12]:
# avg win, avg lose, ratio
avg_win = round(df[df.realizedPNL > 0].realizedPNL.mean())
avg_lose = round(df[df.realizedPNL < 0].realizedPNL.mean())
ratio = round(abs(avg_win / avg_lose), 2)
avg_win, avg_lose, ratio

(1709.0, -514.0, 3.32)

In [13]:
df.realizedPNL.min(), df.realizedPNL.max(), round(abs(df.realizedPNL.max()/df.realizedPNL.min()))

(-1180.0, 6349.0, 5.0)

In [14]:
df.sort_values('realizedPNL').iloc[1]

sys_time        2019-10-31 15:22:51.118069
contract                              NQZ9
action                                SELL
amount                                   1
price                              6941.52
exec_ids                             [366]
order_id                               419
reason                            stop-out
com_exec_id                            366
commission                             1.3
currency                               USD
realizedPNL                        -1119.5
com_sys_time    2019-10-31 15:22:51.118138
Name: 2018-10-25 22:04:00, dtype: object

In [15]:
df.loc['20181011 20:00:00':].head(50)

,sys_time,contract,action,amount,price,exec_ids,order_id,reason,com_exec_id,commission,currency,realizedPNL,com_sys_time
time,,,,,,,,,,,,,
2018-10-11 20:38:00,2019-10-31 15:22:14.043415,NQZ9,SELL,1,6995.750,[325],372,entry,325,1.3,USD,0.0,2019-10-31 15:22:14.043751
2018-10-11 21:16:00,2019-10-31 15:22:14.382266,NQZ9,BUY,1,7035.375,[326],373,stop-out,326,1.3,USD,-792.5,2019-10-31 15:22:14.382336
2018-10-12 08:28:00,2019-10-31 15:22:15.473821,NQZ9,BUY,1,7144.425,[327],374,entry,327,1.3,USD,0.0,2019-10-31 15:22:15.474152
2018-10-12 17:44:00,2019-10-31 15:22:17.020556,NQZ9,SELL,1,7107.825,[328],376,close,328,1.3,USD,-732.0,2019-10-31 15:22:17.020800
2018-10-12 18:07:00,2019-10-31 15:22:17.140126,NQZ9,SELL,1,7092.800,[329],377,entry,329,1.3,USD,0.0,2019-10-31 15:22:17.140460
2018-10-12 21:08:00,2019-10-31 15:22:18.044350,NQZ9,BUY,1,7133.200,[330],378,stop-out,330,1.3,USD,-808.0,2019-10-31 15:22:18.044422
2018-10-12 21:27:00,2019-10-31 15:22:18.160253,NQZ9,BUY,1,7150.825,[331],379,entry,331,1.3,USD,0.0,2019-10-31 15:22:18.160539
2018-10-15 08:14:00,2019-10-31 15:22:19.006325,NQZ9,SELL,1,7113.975,[332],380,stop-out,332,1.3,USD,-737.0,2019-10-31 15:22:19.006395
2018-10-15 16:06:00,2019-10-31 15:22:20.149874,NQZ9,SELL,1,7083.575,[333],381,entry,333,1.3,USD,0.0,2019-10-31 15:22:20.150316


In [16]:
df.realizedPNL.resample('M').sum()

time
2018-02-28     -456.00
2018-03-31    11347.36
2018-04-30    -1839.00
2018-05-31    -4764.00
2018-06-30     2424.94
2018-07-31     1872.50
2018-08-31      130.00
2018-09-30    -1050.84
2018-10-31    -2074.00
2018-11-30     2608.50
2018-12-31    10744.16
2019-01-31     1629.50
2019-02-28     2640.00
2019-03-31     3616.57
2019-04-30     1391.50
2019-05-31    -3418.00
2019-06-30     3494.91
2019-07-31    -1391.50
2019-08-31     2498.00
2019-09-30    -2246.00
Freq: M, Name: realizedPNL, dtype: float64